# Recursive Data Partitioning

In this notebook be analyse an algorithm for recursive data partitioning. Such algorithms may be combined with regression methods to improve approximation accuracy. The method is a variant of [Local Regression](https://en.wikipedia.org/wiki/Local_regression) and [Regression Trees](https://en.wikipedia.org/wiki/Decision_tree_learning).

## Introduction and Notation

We consider a data set $C$ with $m$ observaions and $k$ control variables. The daa set is represented as matrix
$$
  C =
    \left[ \begin{matrix}
      c_{1,1} & \ldots & c_{1,k} \\
      \vdots  &        & \vdots  \\
      c_{m,1} & \ldots & c_{m,k}
    \end{matrix} \right].
$$
We assume that the columns
$$
  C_j = 
    \left[ \begin{matrix}
      c_{1,j}  \\
      \vdots   \\
      c_{m,j} 
    \end{matrix} \right].
$$
 ($j=1,\ldots,k$) can be ordered. This assumption is naturally satisfied for real data points.

We aim at specifying a recursive partitioning of the data set $C$ along the columns $C_1,\ldots,C_k$. The partitioning is specified by an index list
$$
  {\cal N} = \left[ n_1, \ldots, n_k \right].
$$
Each $n_j\geq 1$ represents the number of partitions for the $j$-th control variable. The recursive partitioning yields a total of
$$
  n = \prod_{j=1}^k n_j
$$
partitions of the data set $C$.

## Assigning Data to Partitions

We consider a data point
$$
  D = \left[ d_1, \ldots, d_k \right].
$$
A data point $D$ is mapped to a particular partition $p\in \left\{ 0, \ldots, n-1 \right\}$ via an index function. The index function is decomposed into two steps.

In a first step we assign the data point $D$ to a multi-index
$$
 {\cal I} = \left[ i_1, \ldots, i_k \right].
$$
Each index component is $i_j \in \left\{ 0, \ldots, n_j-1 \right\}$. Details of that mapping are discussed in the forthcomming section.

In a second step we calculate the partition $p$ from the multi-index ${\cal I}$ via
$$
  p = \sum_{j=1}^k i_j \left(\prod_{l=j+1}^k n_l \right).
$$
Indeed, we have $p \in \left\{ 0, \ldots, n-1 \right\}$.

## Multi-Index Mapping

In this section we specify the mapping of a data point $D$ to a multi-index ${\cal I}$.

We define a sequence of branching matrices $Q^{(1)}, \ldots, Q^{(k)}$. Each branching matrix $Q^{(j)}$ is of shape
$$
  Q^{(j)} =
    \left[ \begin{matrix}
      q^{(j)}_{0,1}   & \ldots & q^{(j)}_{0,n_j-1} \\
      \vdots          &        & \vdots           \\
      q^{(j)}_{m_j-1,1} & \ldots & q^{(j)}_{m_j-1,n_j-1}   
    \end{matrix} \right].
$$
The number of rows $m_j$ are given as
$$
  m_j = \prod_{l=1}^{j-1} n_l.
$$
We note that the rows of the matrix $Q^{(j)}$ degenerate if $n_j=1$. This case corresponds to the situation where only a single partition for the observable $C_j$ is specified.

The rows $q^{(j)}_{r}$ ($r=0,\ldots,m_j-1$) are
$$
  q^{(j)}_{r} = \left[ q^{(j)}_{r,1}, \ldots, q^{(j)}_{r,n_j-1} \right].
$$
The elements of each row represent quantiles of the distribution of data points $d_j$. We assume that the elements of $q^{(j)}_{r}$ are strictly monotonically increasing.

For a given row $q^{(j)}_{r}$ and a data point $d_j$ we define the search function $s\left(q^{(j)}_{r},d_j\right)$ as
$$
  s\left(q^{(j)}_{r},d_j\right) = \min\left\{ i-1 \; | \; d_j < q^{(j)}_{r,i}  \right\}.
$$
If $d_j \geq q^{(j)}_{r,n_j-1}$ then $s\left(q^{(j)}_{r},d_j\right) = n_j-1$. Moreover, if $n_j=1$ then $s\left(q^{(j)}_{r},d_j\right) = 0$. With this specification we get
$$
  s\left(q^{(j)}_{r},d_j\right) \in \left\{ 0,\ldots,n_j-1 \right\}.
$$
Such a search function is implemented via Numpy's *searchsorted* method.

The mapping of a data point $D$ to a multi-index ${\cal I}$ is now specified recursively as
$$
  i_j = s\left(q^{(j)}_{r_j},d_j\right)
$$
with
$$
  r_j = \sum_{t=1}^{j-1} i_t \left(\prod_{l=t+1}^{j-1} n_l \right).
$$

## Calibration of Branching Matrices

In this section we specify the calculation of the branching matrices $Q^{(1)}, \ldots, Q^{(k)}$. Input to the calculation procedure is the data set $C$.

We start with $Q^{(1)} = \left[ q^{(1)}_{0,1}, \ldots, q^{(1)}_{0,n_1-1} \right]$. If $n_1=1$ then $Q^{(1)}$ is empty. Otherwise, we set $q^{(1)}_{0,t}$ ($t=1,\ldots,n_1-1$) to the $t/n_1$-quantile of the sample $C_1$.

With $Q^{(1)}$ we can now calculate $i_1$ for each row of $C$.

The calculation of $Q^{(j)}$ for $j=2,\ldots,k$ is now derived recursively. We calculate $r_j$ for each row of $C$. Next we sort and divide the data set $C$ according to $r_j$.

For each $r = 0,\ldots,m_j-1$ we select the rows $\tilde C$ of the original data set $C$ with $r_j=r$. If $n_j=1$ then $q^{(j)}_{r}$ is empty. Otherwise, we set $q^{(j)}_{r,t}$ ($t=1,\ldots,n_j-1$) to the $t/n_j$-quantile of the sample $\tilde C_j$.

With the branching matrix $Q^{(j)}$ we can now also calculate $i_j$ for each row of $C$.

After the calculation of all $i_j$ ($j=1,\ldots,k$) for all rows of $C$ we can then finally calculate the partition index $p$.

## Code

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
partition = lambda I, N : int(np.sum([ i*p for i, p in \
    zip(I, [np.prod(N[j+1:]) for j in range(I.shape[0])]) ]))
"""
Calculate the partition index for a given multi-index.
"""

In [ ]:
def index(d, N, I, Q):
    """
    Calculate the index i for the element d given a partitioning.

    Arguments:
    d -- a scalar data points
    N -- a 1-dim array of number of partitions
    I -- a 1-dim array of indices for elements before d
    Q -- a 2-dim array used as branching matrix
    """
    assert len(N.shape) == 1
    assert len(I.shape) == 1
    assert I.shape[0] == N.shape[0] - 1
    # indices must be consistent with partitions
    for i, n in zip(I,N[:-1]):
        assert 0 <= i
        assert i <  n
    # we must have at least one (last) partition
    assert N[-1] > 0
    # branching matrix must have consistent shape
    m_j = 1  # default for empty N[:-1]
    if N.shape[0]>1:
        m_j = np.prod(N[:-1])
    assert Q.shape == (m_j, N[-1]-1)
    #
    if N[-1] == 1:
        return 0
    r = partition(I,N[:-1])
    i = np.searchsorted(Q[r],d)
    return i

In [ ]:
def branching_matrix(I, C, N):
    """
    Calculate a branching matrix for a data column C and given partitioning.

    Arguments:
    I -- a 2-dim array of indices for all elements before C
    C -- a 1-dim array of data points used for calibration
    N -- a 1-dim array of number of partitions
    """
    assert len(I.shape) == 2
    assert len(C.shape) == 1
    assert len(N.shape) == 1
    assert I.shape == (C.shape[0], N.shape[0] - 1)
    # indices must be consistent with partitions
    for i, n in zip(np.transpose(I),N[:-1]):
        assert np.all(0 <= i)
        assert np.all(i <  n)
    # we must have at least one (last) partition
    assert N[-1] > 0
    # 
    m_j = 1  # default for empty N[:-1]
    if N.shape[0]>1:
        m_j = np.prod(N[:-1])
    if N[-1] == 1:
        Q  = np.zeros([m_j,0])  #  degenerated brnaching matrix
        return Q
    #
    quantiles = np.array([ t/N[-1] for t in range(1,N[-1]) ])  # equal partitions
    P = np.array([ partition(i,N[:-1]) for i in I ])
    Q = np.array([
        np.quantile(C[P==r], quantiles) for r in range(m_j) ])
    return Q

In [ ]:
def calculate_partitioning(C, N):
    """
    Calculate a partitioning for a data set C and partition indices N

    Arguments:
    C -- a 2-dim array of data points used for calibration
    N -- a 1-dim array of number of partitions
    """
    assert len(C.shape) == 2
    assert len(N.shape) == 1
    assert C.shape[1] == N.shape[0]
    assert np.all(N > 0)
    I = np.zeros([C.shape[0],0])
    Q_list = []
    for j in range(N.shape[0]):
        #print(j)
        Q_j = branching_matrix(I,C[:,j],N[:j+1])
        #print(Q_j)
        I_j = np.array([ index(c, N[:j+1], i, Q_j) for c, i in zip(C[:,j],I) ])
        #print(I_j)
        I_j.shape = (I_j.shape[0], 1)
        I = np.append(I, I_j, axis=1)
        Q_list += [Q_j]
    P =  np.array([ partition(i,N) for i in I])
    return I, Q_list, P

In [ ]:
def multi_index(d, N, Q_list):
    """
    Calculate the multi-index for a data set d and a given list of branching matrices.

    Arguments:
    d -- a 1-dim array representing a data point
    N -- a 1-dim array of number of partitions
    Q_list -- list of branching matrices
    """
    assert len(d.shape) == 1
    assert len(N.shape) == 1
    assert d.shape[0] == N.shape[0]
    assert d.shape[0] == len(Q_list)
    I = np.zeros([0])
    for j in range(d.shape[0]):
        i = index(d[j], N[:j+1], I, Q_list[j])
        I = np.append(I,i)
    return I

## Testing

We start by creating a test data set.

In [ ]:
rg = np.random.Generator(np.random.PCG64(123))
C = rg.standard_normal([1000,3])
C.shape

In [ ]:
N = np.array([3,2,3])
I, Q_, P = calculate_partitioning(C,N)
Q_

In [ ]:
multi_index(C[3],N,Q_)

In [ ]:
I[3]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for j in [0,1,2]:
    plt.figure()
    plt.plot(P,C[:,j],'.',label='C[%s]' % j)
    plt.legend()

## Application for Regression

In [ ]:
import sys
sys.path.append('../')  # make python find our modules
from hybmc.mathutils.Regression import Regression

In [ ]:
class PieceWiseRegression:
    """
    Split regressions based on break point.

    For details see https://en.wikipedia.org/wiki/Segmented_regression
    """

    def __init__(self, controls, observations, \
        maxPolynomialDegree=2, partitioning=None):
        self.maxPolynomialDegree = maxPolynomialDegree
        self.partitioning = partitioning
        if self.partitioning is None:    # default
            self.partitioning = np.array([ 1 for j in range(controls.shape[1]) ])
        #
        I, Q_list, P = calculate_partitioning(controls, self.partitioning)
        self.Q_list = Q_list
        self.regressions = []
        for r in range(np.prod(self.partitioning)):
            self.regressions.append(Regression(controls[P==r], observations[P==r], \
                self.maxPolynomialDegree))

    def value(self, control):
        I = multi_index(control, self.partitioning, self.Q_list)
        r = partition(I, self.partitioning)
        v = self.regressions[r].value(control)
        return v


## Black-Scholes Example

In [ ]:
S0 = 1.0
sigma = 0.15
times = np.array([0.0, 1.8, 2.0])
nPaths = 2**13
rg = np.random.Generator(np.random.PCG64(123))
dW = rg.standard_normal([nPaths,times.shape[0]-1])
X = np.zeros([nPaths,times.shape[0]])
for k in range(times.shape[0]-1):
    dt = times[k+1]-times[k]
    X[:,k+1] = X[:,k] - 0.5*sigma**2 * dt + (sigma*np.sqrt(dt)) * dW[:,k]
S = S0 * np.exp(X)
V_T = np.maximum(S[:,2] - S0, 0.0)

In [ ]:
from hybmc.mathutils.Helpers import Black
V_t = Black(S0, S[:,1], sigma, times[2]-times[1], 1.0)

In [ ]:
d = 2
p = np.array([3])
regression = PieceWiseRegression(S[:,1:2], V_T, d, p)
R_t = np.array([ regression.value(c) for c in S[:,1:2] ])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(S[:,1], V_t, 'r.', label='Black')
plt.plot(S[:,2], V_T, 'b.', label='Payoff')
plt.legend()

In [ ]:
plt.plot(S[:,1], V_t, 'r.', label='Black')
plt.plot(S[:,1], R_t, 'g.', label='Regression')
plt.legend()